In [33]:
import imageio
import numpy as np
from scipy import ndimage

In [30]:
sec1TiffPaths = [
    './sec1/tiff/1.tiff',
    './sec1/tiff/2.tiff',
    './sec1/tiff/3.tiff'
]

In [31]:
sec1JpegPaths = [
    './sec1/jpg/1.jpg',
    './sec1/jpg/2.jpg',
    './sec1/jpg/3.jpg'
]

In [19]:
sec1ExpTimes = [
    0.03333,
    0.125,
    0.5
]

In [20]:
sec1Tiffs = map(lambda path: imageio.imread(path) / 65536.0, sec1TiffPaths)

In [34]:
sec1Jpegs = map(lambda path: ndimage.imread(path) / 256.0, sec1JpegPaths)

In [22]:
def w(images):
    result = []
    
    for i in range(len(images)):
        wMatrix = np.exp(-4 * ((images[i] - 0.5) ** 2) / (0.5 ** 2))
        result.append(wMatrix)
        
    return result

In [35]:
sec1TiffWeights = w(sec1Tiffs)

In [36]:
sec1JpegWeights = w(sec1Jpegs)

In [26]:
def robHDR(images, weights, expTimes):
    up = np.zeros(images[0].shape)
    down = np.zeros(images[0].shape)
    
    for i in range(len(images)):
        up += weights[i] * expTimes[i] * images[i]
        down += weights[i] * (expTimes[i] ** 2)
    
    result = up / down
    
    return result

In [27]:
sec1TiffHDR = robHDR(sec1Tiffs, sec1TiffWeights, sec1ExpTimes)

In [37]:
sec1JpegHDR = robHDR(sec1Jpegs, sec1JpegWeights, sec1ExpTimes)

In [28]:
def saveHDR(filename, image):
    f = open(filename, "wb")
    f.write("#?RADIANCE\n# Made with Python & Numpy\nFORMAT=32-bit_rle_rgbe\n\n")
    f.write("-Y {0} +X {1}\n".format(image.shape[0], image.shape[1]))
    
    brightest = np.maximum(np.maximum(image[...,0], image[...,1]), image[...,2])
    mantissa = np.zeros_like(brightest)
    exponent = np.zeros_like(brightest)
    np.frexp(brightest, mantissa, exponent)
    scaled_mantissa = mantissa * 256.0 / brightest
    rgbe = np.zeros((image.shape[0], image.shape[1], 4), dtype=np.uint8)
    rgbe[...,0:3] = np.around(image[...,0:3] * scaled_mantissa[...,None])
    rgbe[...,3] = np.around(exponent + 128)
    
    rgbe.flatten().tofile(f)
    f.close()

In [29]:
saveHDR('./sec1/tiff/result.hdr', sec1TiffHDR)

In [38]:
saveHDR('./sec1/jpg/result.hdr', sec1JpegHDR)